# Appendix 1: Data Cleaning

In [1]:
import re
import bs4 #this may be appendices only
import requests #this may be appendices only
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from datetime import datetime
import datetime
import seaborn as sns
from   sklearn.linear_model import LinearRegression, LogisticRegression
from   sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

In [10]:
#ingest LibraryReads archive

libraryreads = pd.read_excel("LibraryReadsMasterList.xlsx")
#libraryreads=libraryreads.drop(libraryreads.loc[:,'Unnamed: 13':'Unnamed: 23'].columns, axis=1)

#rename columns
orig_lr_col = libraryreads.columns
new_lr_col = [i.lower().replace(' ','_') for i in orig_lr_col]
libraryreads.columns=new_lr_col
libraryreads=libraryreads.rename({'list_date':'lr_my','author_(last,_first)':'author','top_pick':'lr_top', 'hall_of_fame':'lr_hof','year-end_favorite_list':'lr_yfav','nominator/library':'nom_lib_lr'}, axis="columns")

#review rows without title (likely header rows for each month's list)
libraryreads[libraryreads['title'].isna()==True].to_csv('LibraryReads_NoTitle.csv',)
#confirmed that all rows without a title are header only
##Note: December 2016/January 2017 were combined into a single list, but it doesn't appear additional books were nominated/included (verified)

libraryreads[libraryreads['title'].isna() == False].describe
libraryreads=libraryreads[libraryreads['title'].isna()==False].reset_index() #remove month header rows and reset index


#Find bad isbn numbers - there are some with alphabet characters and one with internal spaces
libraryreads['isbn'] = libraryreads['isbn'].astype('string')
libraryreads['isbn']=libraryreads.isbn.str.strip().tolist()
libraryreads.isbn.str.isdigit().value_counts()
libraryreads[libraryreads.isbn.str.isdigit()==False]

#update ISBN numbers for those 4 rows to be all digits, to transform col into int
#index 127, Party of 2 - use hardcover IBSN, not trade paperback
libraryreads.iloc[127,6]='9780593100813'
libraryreads.iloc[127,6]
#index 246 and 336: strip out '-'
libraryreads.iloc[246,6]=libraryreads.iloc[246,6].replace('-','')
libraryreads.iloc[311,6]=libraryreads.iloc[311,6].replace('-','')
#index 529 - should be 0, not o
libraryreads.iloc[529,6]='9780385534246'

libraryreads[libraryreads.isbn.str.len() != 13]

#ISBN from booklist, corroborated by NoveList: 9780062460226
libraryreads.iloc[527,6]='9780062460226'

pub_date_fix_list=[] #fix type of pub_date column
for i in libraryreads.index:
    if isinstance(libraryreads.pub_date[i],datetime.datetime)==False:
        pub_date_fix_list.append(i)
#        print(i,libraryreads.pub_date[i], type(libraryreads.pub_date[i]))

libraryreads.iloc[21]
#pubdate for The Kindest Lie: 2/2/2021
libraryreads.iloc[21,4] = datetime.datetime.strptime('02/02/2021','%m/%d/%Y')

#index 353 looks fine from here: 11/28/2018
libraryreads.iloc[353,4] = datetime.datetime.strptime(libraryreads.pub_date[353],'%m/%d/%Y')
#Handful of May dates were entered incorrectly - 3 digits for the day, 1 for the month.
pub_date_fix_list=[]
for i in libraryreads.index:
    if isinstance(libraryreads.pub_date[i],datetime.datetime)==False:
        pub_date_fix_list.append(i)
#        print(i,libraryreads.pub_date[i], type(libraryreads.pub_date[i]))


for i in pub_date_fix_list:
    libraryreads.iloc[i,4]=datetime.datetime.strptime('0'+libraryreads.pub_date[i][0:2]+libraryreads.pub_date[i][3:],'%m/%d/%Y')

libraryreads['isbn']=pd.to_numeric(libraryreads['isbn'])
libraryreads=libraryreads.drop(libraryreads.loc[:,'unnamed:_13':'unnamed:_23'].columns, axis=1)
libraryreads['isbn'].to_csv('LibraryReads_isbn.csv') #output ISBN list to CSV for manual search in the LC catalog

C:\Users\Aryn\anaconda3\envs\fresh\lib\site-packages\openpyxl\worksheet\_reader.py:206: UserWarning: Cell D25 is marked as a date but the serial value 6684337 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


In [11]:
#Get LOC Permalinks from email text
#ISBNs entered into the advanced search in LC Catalog in groups of 85 - 170. Results can be emailed/exported in batches of 100
#most searches resulted in less than 150 records
#text formatted results from each export were copied into a single text document for a round of searching, and then the text processed to retrieve the permalinks to the LC records
#Multiple rounds of searches were run. The first round resulted in approximately 450 non-successes, the second in slightly less than 100 non-successes, and the third resulted in only 80 unmatched records.

loc_email=open("LOC Email Data.txt",encoding='utf8')
print(loc_email)
link_lst1 = []

for ln in loc_email:
    if ln.lstrip().startswith('Permalink'): #find permalink rows of text
        link_lst1.append(ln.lstrip().rstrip('\n')[12:]) #remove "Permalink: " from the string, retain only the actual link

loc_email.close()

loc_email=open("LOC Email Data2.txt",encoding='utf8')
print(loc_email)
link_lst2 = []

for ln in loc_email:
    if ln.lstrip().startswith('Permalink'):
        link_lst2.append(ln.lstrip().rstrip('\n')[12:])

loc_email.close()

loc_email=open("LOC Email Data3.txt",encoding='utf8')
print(loc_email)
link_lst3 = []

for ln in loc_email:
    if ln.lstrip().startswith('Permalink'):
        link_lst3.append(ln.lstrip().rstrip('\n')[12:])
loc_email.close()

link_lst=list(set(link_lst1 + link_lst2 + link_lst3))#concatenate all lists of links into a single list to loop over for webscraping.
len(link_lst)

<_io.TextIOWrapper name='LOC Email Data.txt' mode='r' encoding='utf8'>
<_io.TextIOWrapper name='LOC Email Data2.txt' mode='r' encoding='utf8'>
<_io.TextIOWrapper name='LOC Email Data3.txt' mode='r' encoding='utf8'>


1003

In [ ]:
#Not executed for phase IV version of data cleaning due to run time
outer_loop_results = []
link_lst= list(set(link_lst)) #remove duplicate permalinks, as each search round may have resulted in overlaps
k_range = len(link_lst)
for k in range(0,k_range):
    loc_req = requests.get(link_lst[k])
    soup_hold = bs4.BeautifulSoup(loc_req.text,'html.parser')
    item_title_hold=soup_hold.findAll('h3',attrs={'class':'item-title'}) #class names are very limited on the LC catalog pages, which offers flexibility in webpage creation, but makes specifying a particular item title difficult.
    title_genre_list=[] #the form/genres associated with an LC record can be an n-dimensional array 
    results_holder = [] #not used in the final version of this loop, temporary step in loop development
    for i in range(0,len(item_title_hold)):
        item_title_i = item_title_hold[i]
        if item_title_i.string=='ISBN': #find the ISBN item-title
            parent_div = item_title_i.parent #look at all parts in the associated div - due to webpage structure, the actual value of the ISBN isn't stored with the ISBN title
            isbn_field = parent_div.h3.next_sibling.next_sibling.li.span.string[0:13] #two siblings below the ISBN is the <span> tag that contains the ISBNs associated with the record. This code chooses the first ISBN and assumes it is an ISBN 13
        if item_title_i.string == 'Form/Genre':
            parent_div = item_title_i.parent
            for j in parent_div.findAll('span'):
                title_genre_list.append(j.string)
            results_holder.append((isbn_field,title_genre_list)) #this will be emptied out for each iteration of the outer loop. 
    outer_loop_results.append((link_lst[k],isbn_field,title_genre_list))
print('Done')
loc_results = pd.DataFrame(outer_loop_results, columns = ['permalink','isbn','genres'])
loc_results.head()

#Find cases where the ISBN 13 assumption is invalid by identifying cases where a non-numeric character is in the isbn field

loc_results = pd.DataFrame(outer_loop_results, columns = ['permalink','isbn','genres'])
loc_results.head()
loc_results.isbn.astype(str)
loc_results[loc_results.isbn.str.isdigit()==False]

loc_results.to_csv('loc_results_v2.csv')

In [23]:
loc_results = pd.read_csv('loc_results_v2.csv') #read in processed results rather than repeat webscraping for Phase IV
loc_results = loc_results.drop(columns=['Unnamed: 0'])

In [24]:
loc_results['isbn']=loc_results['isbn'].str.rstrip() 
loc_results.dtypes
loc_results['isbn'] = loc_results['isbn'].astype('string')
loc_results['isbn']=loc_results.isbn.str.strip().tolist()
#print(loc_results.isbn.str.isdigit().value_counts()) #check if removing leading or trailing spaces rectifies any of the non-integer ISBN values (no anticipated successes)
loc_results[loc_results.isbn.str.isdigit()==False]

#ISBN 10, not ISBN 13 was first ISBN listed in Library of Congress Data. Replace with the ISBN 13 value.
loc_results.iloc[148,1]=9780451495587
loc_results.iloc[458,1]=9780316243391
loc_results.iloc[747,1]=9780062694058
loc_results.iloc[960,1]=9780062132529
loc_results[loc_results.isbn.str.isdigit()==False] #verify replacement worked

,permalink,isbn,genres


In [27]:
loc_results['isbn']=pd.to_numeric(loc_results['isbn'])
loc_results['genres']=loc_results.apply(lambda row: list(set(row['genres'])), axis = 1) #remove duplicate form/genre classifications within a list associated with a single ISBN
loc_results.columns


Index(['permalink', 'isbn', 'genres'], dtype='object')

Code has broken - importing full wide dataset below, will correct for phase V.

In [ ]:
loc_results.genres.to_list()
genres_split = pd.DataFrame(loc_results.genres.to_list(),index = loc_results.index) #create a dataframe that splits the genre list of length n into n columns
genres_split
loc_results = pd.concat([loc_results,genres_split],axis=1) #merge genre dataframe with existing Library of Congress Dataframe
loc_results = loc_results.rename({0:'genre_0', 1:'genre_1',2:'genre_2',3:'genre_3',4:'genre_4',5:'genre_5',6:'genre_6',7:'genre_7'}, axis=1) #rename genre column headers
loc_dedup = loc_results.drop(columns='genres').drop_duplicates() #drop_duplicates generates an error when one of the columns is a list, drop the genre list in favor of retaining only the genre columns

In [16]:
full_df = pd.merge(libraryreads,loc_dedup,how='left',on='isbn') #merge datasets
#939 rows, but 936 distinct ISBNs in the libraryReads data
isbn_mult = pd.DataFrame(full_df['isbn'].value_counts())
multiple_row_isbn = list(isbn_mult[isbn_mult.isbn==2].index)

#look at all records for ISBNs that appear multiple times in the data
for i in multiple_row_isbn:
    print(full_df[full_df.isbn==i])
#isbn = 9781616149987 - first record is edition with 14 page preview of forthcoming novel. No genres listed in either record. Retain row 801 (no additional pages)
#isbn = 978006228259 -  retain record in row 769 - multiple publishers listed with the correct pub year, row 769 corresponds with publisher named in LibraryReads. Row 770 is the large print edition. 
#isbn = 9781476705996 - retain line 899. Visual inspection doesn't suggest any reason to prefer permalink over the other. 

full_df = full_df.drop([800,770,900])
#confirm no more duplicate ISBNs
isbn_mult = pd.DataFrame(full_df['isbn'].value_counts())
multiple_row_isbn = list(isbn_mult[isbn_mult.isbn==2].index)

for i in multiple_row_isbn:
    print(full_df[full_df.isbn==i])
print('none')


     index    lr_my         author                             title  \
899    980  2013/12  Harper, Molly  How to Run With a Naked Werewolf   
900    980  2013/12  Harper, Molly  How to Run With a Naked Werewolf   

                pub_date     publisher           isbn lr_top lr_hof debut  \
899  2013-12-31 00:00:00  Pocket Books  9781476705996    NaN    NaN   NaN   
900  2013-12-31 00:00:00  Pocket Books  9781476705996    NaN    NaN   NaN   

     ...    23    24    25    26    27    28    29    30    31    32  
899  ...  None  None  None  None  None  None  None  None  None  None  
900  ...  None  None  None  None  None  None  None  None  None  None  

[2 rows x 48 columns]
     index    lr_my         author              title             pub_date  \
800    872  2014/10  Eskens, Allen  Life We Bury, The  2014-10-14 00:00:00   
801    872  2014/10  Eskens, Allen  Life We Bury, The  2014-10-14 00:00:00   

                publisher           isbn lr_top lr_hof debut  ...    23    24  \

In [20]:
print(len(full_df.columns))

48


In [62]:
#clean up publisher information by removing formatting differences
full_df['publisher']=full_df['publisher'].astype(str)
full_df['publisher']=full_df['publisher'].str.lower()
full_df['publisher']=full_df['publisher'].str.rstrip()
full_df['publisher']=full_df['publisher'].str.lstrip()
#Cleaning publisher names by visual inspection and validation
pd.set_option('display.max_rows',140)
full_df['publisher'].value_counts().sort_index()

ace                                 7
algonquin books                    14
algonuin                            1
amistad                             1
amy einhorn/putnam                  1
atlantic monthly press              3
atria                               2
atria books                        17
atria/37 ink                        1
atria/emily bestler                 2
avon                               23
ballantine                          2
ballantine books                   34
ballatine books                     1
bantam                              7
beacon press                        1
bellevue literary press             1
berkley                            70
berkley crime prime                 1
berkley jove                        7
berkley prime crime                 1
berkley/jove                        5
bloomsbury                          2
bloomsbury publishing               2
bloomsbury usa                      2
blue rider press                    1
broadway boo

In [63]:
#Publisher clean up
#print(full_df[full_df.publisher=='algonuin']) #confirmed typo
full_df.loc[505,'publisher']='algonquin books'
full_df[(full_df.publisher.str.contains('atria'))] #atria is the same as atria books, /xxx indicates a publisher within the umbrella of Atria
full_df.loc[417,'publisher']='atria books'
full_df.loc[513,'publisher']='atria books'
full_df[(full_df.publisher=='ballatine books') | (full_df.publisher=='ballantine')] #confirmed both are synonyms for ballantine books

,index,lr_my,author,title,pub_date,publisher,isbn,lr_top,lr_hof,debut,...,23,24,25,26,27,28,29,30,31,32
321,347,2019/02,"Letts, Elizabeth",Finding Dorothy: A Novel,2019-02-12 00:00:00,ballatine books,9780525622109,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
363,392,2018/10,"Picoult, Jodi",A Spark of Light: A Novel,2018-10-02 00:00:00,ballantine,9781984828095,NaN,x,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
803,874,2014/10,"Picoult, Jodi",Leaving Time,2014-10-14 00:00:00,ballantine,9780345544926,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None


In [64]:
def publisher_replace(publishername,*row): #create a function for updates after gaining a better understanding of how many updates may be required
    '''
    replace the name of a publisher for given rows within the more common name from the dataset
    '''
    full_df.loc[row,'publisher']=publishername
    #print('complete')
def publisher_find(namev1,namev2):
    print(full_df[(full_df.publisher==namev1) | (full_df.publisher==namev2)])

In [65]:
publisher_find('berkley prime crime', 'berkley crime prime') #berkley prime crime matches LC record
publisher_find('berkley jove','berkley/jove')#LC records vary within both of these. Use berkley/jove, more consistent with format from atria imprints
print(full_df[full_df['publisher'].str.contains('bloomsbury')])
publisher_find('doubleday books','')
full_df[full_df['publisher'].str.contains('giroux')]#LC lists separately, do not combine
publisher_find('harlequin mira','mira')#Harlequin MIRA is in the LC, confirmed on Harlequin publishers as MIRA. 
publisher_find('hqn','hqn books') #majority are hqn books, no Library of Congress records to use for validation/comparison
publisher_find('little brown','little, brown and company')#little, brown and company
publisher_find('mcd','')#mcd/Farrar, Strauss & Giroux
publisher_find('minotaur','minotaur books')
publisher_find('park row', 'park row books')
publisher_find('pegasus','pegasus books')
print(full_df[full_df.publisher.str.contains('penguin')]) #Penguin books and penguin press are both found in LC, penguin should be penguin books by LC link
publisher_find('putnam','putnam adult') #these should all go to GP Putnam's sons
publisher_find('quirk','quirk books') #these go to quirk books
publisher_find('riverhead books','riverhead')
publisher_find('simon and schuster','')
publisher_find('soho press','sourcebooks/landmark')
print(full_df[full_df.publisher.str.contains('tor')])
publisher_find('viking','viking adult')


     index    lr_my          author                               title  \
372    402  2018/09  McKinlay, Jenn                   Hitting the Books   
376    407  2018/08     Bowen, Rhys  Four Funerals and  Maybe a Wedding   

                pub_date            publisher           isbn lr_top lr_hof  \
372  2018-09-11 00:00:00  berkley prime crime  9780451492678    NaN    NaN   
376  2018-08-07 00:00:00  berkley crime prime  9780425283523    NaN    NaN   

    debut  ...    23    24    25    26    27    28    29    30    31    32  
372   NaN  ...  None  None  None  None  None  None  None  None  None  None  
376   NaN  ...  None  None  None  None  None  None  None  None  None  None  

[2 rows x 92 columns]
     index    lr_my                 author  \
2        3  2021/03            Desai, Sara   
37      40  2021/01   Solomon, Rachel Lynn   
157    170  2020/03       Adams, Lyssa Kay   
204    221  2019/11       Adams, Lyssa Kay   
206    223  2019/11  Christopher, Andie J.   
258    27

In [67]:
publisher_replace('ballantine books',803,321,363)
publisher_replace('berkley prime crime',376)
publisher_replace('berkley/jove',2,37,157,204,206,258,285)
publisher_replace('bloomsbury publishing',33,83,302,325,595,741)
publisher_replace('doubleday',217)
publisher_replace('harlequin mira',196,934)
publisher_replace('hqn books',22)
publisher_replace('little, brown and company',213,266,386,420,464,584,621,674,732,736,759,788,831,841,859,866,928,932)
publisher_replace('mcd/farrar, straus & giroux',407,484)
publisher_replace('minotaur books',398,497)
publisher_replace('park row books',9,25,50,79,172,182,278,310,313,368,480,523)
publisher_replace('pegasus books',640,708,790)
publisher_replace('penguin books',795)
publisher_replace('g.p. putnam\'s sons',539,548,760,776,872)
publisher_replace('quirk books',38,148,416,812,837)
publisher_replace('riverhead books',54,121,143,191,241,273,406,425,511,577,701,781,817,929)
publisher_replace('simon & schuster',72,129,362)
publisher_replace('sourcebooks landmark',667)
publisher_replace('tor.com',32,66,145)
publisher_replace('tor books',35,78,104,272,287,337,454,510,641,774,825,904)
publisher_replace('viking',809,821)

In [28]:
#Determine how many LibraryReads rows do not have Library of Congress data, proxied with the existence of a permalink. 
full_df['permalink'].describe()
full_df.loc[183]
#4th pass: few results, manual input more efficient than webscraping
full_df.loc[47,'permalink'] = 'https://lccn.loc.gov/2020015957'
full_df.loc[47,'genre_0'] = 'Biographical fiction.'
full_df.loc[183,'permalink'] = 'https://lccn.loc.gov/2020275791'
full_df.loc[183,'genre_0'] = 'Chick lit.'
full_df.loc[183,'genre_1'] = 'Fiction.'
full_df.loc[183,'genre_2'] = 'Humorous fiction'
full_df.loc[183,'genre_3'] = 'Romance fiction'
full_df.loc[183,'genre_4'] = 'Love stories'
full_df.loc[516,'permalink'] = 'https://lccn.loc.gov/2017299737'
full_df.loc[516,'genre_0'] = 'Paranormal romance stories.'
full_df.loc[516,'genre_1'] = 'Fantasy fiction.'
full_df.loc[516,'genre_2'] = 'Fiction.'
full_df.loc[516,'genre_3'] = 'Romance fiction.'
full_df.loc[516,'genre_4'] = 'Love stories.'
full_df.loc[231,'permalink'] = 'https://lccn.loc.gov/2019010863'
full_df.loc[231,'genre_0'] = 'Love stories.'
full_df.loc[708,'permalink'] = 'https://lccn.loc.gov/2015298168'
full_df.loc[708,'genre_0'] = 'Essays.'
full_df.loc[633,'permalink'] = 'https://lccn.loc.gov/2015025112'
full_df.loc[633,'genre_0'] = 'Psychological fiction.'
full_df.loc[633,'genre_1'] = 'Suspense fiction.'
full_df.loc[834,'permalink'] = 'https://lccn.loc.gov/2013048524'
full_df.loc[834,'genre_0'] = 'Love stories.'
full_df.loc[310,'permalink'] = 'https://lccn.loc.gov/2019297682'
full_df.loc[310,'genre_0'] = 'Fiction.'
full_df.loc[325,'permalink'] = 'https://lccn.loc.gov/2018276149'
full_df.loc[325,'genre_0'] = 'Fantasy fiction.'
full_df.loc[325,'genre_1'] = 'Science fiction.'

Remaining data cleaning in Working Analysis notebook: bug in code associated with possible update to pd.read_excel, need to investigate further. 